In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import networkx as nx
import random
import scipy as sp

### Logistic Map

We considered $N$ logistic maps coupled as:

$x_i(t+1) = (1- \epsilon) f[x_i(t)] + \frac{\epsilon}{b_i} \sum_{j=1}^N \eta_{ij} f[(x_j (t-\tau_{ij})]$ 

$t$: discrete time index

$i$: discrete space index

$f(x) = ax(1-x)$: logistic map equation

$\eta_{ij}$: adjacency matrix

$b_i = \sum_j \eta_{ij}$

$\tau_{ij}$: delay time between the $i$th and $j$th

$\epsilon$: coupling strength

#### Network Connectivity

$\langle b \rangle = \frac{1}{N} \sum_{i=1}^N b_i$

In this case we measured the connectivity in terms of the average degree.

#### Synchronization indicators

$\sigma^2 = \frac{1}{N} \left \langle \sum_i (x_i - \langle x\rangle_s)^2 \right \rangle_t $

$\sigma '^2 = \frac{1}{N} \left \langle \sum_i (x_i - x_0)^2 \right \rangle_t $

In [3]:
class Logistic:
    def __init__(self, epsilon = 0.5, t_0 = int(0), T = int(100), n_nodes = None,
                 av_degree = 8, x0 = 1.3, tau_zero = int(1), type_of_graph = 1):
        '''
        PARAMETERS:
        epsilon: float
        t_0: float
        T:float
           Total time of simulated activity
        n_nodes: int
            Number of logistic maps
        av_degree: int
            Average degree of the graph
        x0: float
            Initial state of the logistic map
        tau_zero: int
            Delay parameter
        type_of_graph: int
            If 1, then the graph will follow the Barabasi-Albert
            model. If it's another number it will be Small-World.
        '''
        if n_nodes is None:
            raise ValueError("n_nodes must be specified")
        else:
            self.n_nodes = n_nodes
        self.epsilon = epsilon
        self.t_0 = t_0
        self.T = T
        self.av_degree = av_degree
        self.x0 = [x0]
        self.tau_zero = tau_zero
        self.type_of_graph = type_of_graph
        
    def logistic(self, x, a = 0.5):
        return x*a*(1-x)
    
    def generate_graph(self):
        if self.type_of_graph == 1:
            m = int(self.av_degree/2) 
            self.G = nx.barabasi_albert_graph(self.n_nodes,m) 
        else: 
            k = int(self.av_degree/2)
            p = 1 #probability of rewiring
            self.G = nx.watts_strogatz_graph(self.n_nodes, k, p, seed=None)
        return self.G
    
    def adjacency_matrix(self):
        self.M = nx.to_numpy_array(self.generate_graph())
        return self.M
    
    def calculate_b(self,index):
        return np.sum(self.M[index])
    
    def Tau_matrix(self):
        self.tau_matrix = []
        for i in range(0,self.n_nodes):
            tau = []
            for j in range(0,self.n_nodes):
                tau.append(int(self.tau_zero + np.random.uniform(0,1)))
            self.tau_matrix.append(tau)
        return self.tau_matrix
    
    def interation(self):
        self.M = self.adjacency_matrix()
        self.tau_matrix = self.Tau_matrix()
        self.mapas = {}
        for i in range(0,self.n_nodes):
            self.mapas[i] = self.x0 #[self.logistic(np.random.uniform(0,2))] #Definindo os valores iniciais dos mapas
            
        for t in range(1,self.T): #Percorre nos intervalos de tempo
            for i in range(0,self.n_nodes): #Percorre em todos os nós
                x_T = (1-self.epsilon)*self.logistic(self.mapas[i][t-1])
                soma = []
                for j in range(0,self.n_nodes): #Calcula o somatorio pra cada um dos nós
                    soma.append((self.epsilon/self.calculate_b(i))*self.M[i][j]*
                                self.logistic(self.mapas[j][t-1] - self.tau_matrix[i][j]))
                x_T = x_T + np.sum(soma)
                self.mapas[i].append(x_T)
        return self.mapas
        
    def run(self):
        return self.interation()
    
    def Sigma(self):
        space_mean = []
        for i in range(self.n_nodes): #Calculando a media espacial 
            space_mean.append(self.mapas[i][self.T-1])
        space_mean = np.mean(space_mean)
        sigma = []
        for i in range(n_nodes): #Somatorio 
            k = (self.mapas[i][self.T-1]-space_mean)**2
            sigma.append(k)
        sigma = sum(sigma)
        return (1/self.n_nodes)*(sigma/self.T) #Retorna a média temporal do somatorio divido por n_nodes
    
    def Sigma_line(self):
        sigma_line = []
        for i in range(n_nodes):
            k = (self.mapas[i][self.T-1]- self.mapas[i][0])**2
            sigma_line.append(k)
        sigma_line = sum(sigma_line)
        return (1/self.n_nodes)*(sigma_line/self.T)

In [8]:
T = 100
n_nodes = 40
log = Logistic(epsilon = 1, T = T, n_nodes = n_nodes, av_degree = 5, x0 = 1.3, tau_zero = 1, type_of_graph=1)
mapas = log.run()

In [9]:
print("Sigma:",log.Sigma())
print("Sigma':",log.Sigma_line())

Sigma: 0.0
Sigma': 0.046139577001562494
